In [1]:
# !pip install imbalanced-learn

In [2]:
import numpy as np
import pickle
import paths
import os
import torch

In [3]:
PATHS=paths.create_paths()
print('\n'.join(PATHS))

..\data
..\models
..\logs
..\data\raw_data
..\data\processed_data
..\data\raw_data\states
..\data\raw_data\actions


In [4]:
## Split the states and actions based on actions (This is to )

In [5]:
import imblearn

In [6]:
states_space=[]
actions_space=[]

In [7]:
sum=0
for state, action in zip(os.listdir(PATHS[-2]),os.listdir(PATHS[-1])):
    with open(os.path.join(PATHS[-2],state),'rb') as f:
        state_sample=pickle.load(f)
    with open(os.path.join(PATHS[-1],action),'rb') as f:
        action_sample=pickle.load(f)
    assert len(state_sample[:-5])==len(action_sample[1:-4])
    sum+=len(state_sample[:-5])
    for state_instance, action_instance in zip(state_sample[:-5], action_sample[1:-4]):
        states_space.append(state_instance)
        actions_space.append(action_instance)
    del state_sample, action_sample
print(sum)            

55815


In [8]:
assert len(states_space)==len(actions_space)

Original sum: 55932

In [9]:
state_shape=states_space[0].shape
state_shape

torch.Size([4, 137, 345])

In [10]:
states_shape_reshape=[x.reshape(-1).numpy().astype(np.byte) for x in states_space]

In [11]:
sum_bytes=0
for item in states_shape_reshape:
    sum_bytes+=item.nbytes
print(sum_bytes/(1024**3))
del sum_bytes

9.827673342078924


In [12]:
np.array(states_shape_reshape).nbytes

10552383900

In [13]:
del states_space

In [14]:
for state in states_shape_reshape:
    if len(state.shape)>2:
        print(len(state.shape))
for action in actions_space:
    if type(action) is not int:
        print(type(action))

In [15]:
oversample=imblearn.over_sampling.SMOTE()

In [16]:
%%time
states_shape_reshape,actions_space=oversample.fit_resample(states_shape_reshape,actions_space)

CPU times: total: 45min 9s
Wall time: 48min 58s


In [18]:
with open('D://states.pkl','wb') as f:
    pickle.dump(states_shape_reshape,f)

In [19]:
with open('D://actions.pkl','wb') as f:
    pickle.dump(actions_space,f)

In [20]:
len(states_shape_reshape)

139347

In [ ]:
np.sum(np.array(states_shape_reshape))

#### MANUAL AUGMENTATION

In [ ]:
# sum=0
# states_0, states_1, states_2=[],[],[]
# actions_0, actions_1, actions_2=[],[],[]
# for state, action in zip(os.listdir(PATHS[-2]),os.listdir(PATHS[-1])):
#     with open(os.path.join(PATHS[-2],state),'rb') as f:
#         state_sample=pickle.load(f)
#     with open(os.path.join(PATHS[-1],action),'rb') as f:
#         action_sample=pickle.load(f)
#     assert len(state_sample)==len(action_sample)
#     sum+=len(state_sample)
#     for state_instance, action_instance in zip(state_sample, action_sample):
#         if action_instance==0:
#             states_0.append(state_instance)
#             actions_0.append(action_instance)
#         elif action_instance==1:
#             states_1.append(state_instance)
#             actions_1.append(action_instance)
#         elif action_instance==2:
#             states_2.append(state_instance)
#             actions_2.append(action_instance)
#         else:
#             print('Error in file:',action)
            

In [ ]:
sum

In [ ]:
assert len(states_0)==len(actions_0)
assert len(states_1)==len(actions_1)
assert len(states_2)==len(actions_2)
print(len(actions_0))
print(len(actions_1))
print(len(actions_2))

In [ ]:
s_0=torch.stack(states_0,dim=0)
print(s_0.shape)
s_1=torch.stack(states_1,dim=0)
print(s_1.shape)
s_2=torch.stack(states_2,dim=0)
print(s_2.shape)

In [ ]:
x=torch.vstack(states_1)
print(x.shape)

In [ ]:
import torch

In [ ]:
s_1_1=torch.repeat_interleave(s_1,round(s_0.shape[0]/s_1.shape[0]),dim=0)

In [ ]:
s_2_1=torch.repeat_interleave(s_2,round(s_0.shape[0]/s_2.shape[0]),dim=0)

In [ ]:
actions_1[0]

In [ ]:
a_1_1=torch.repeat_interleave(torch.tensor(actions_1,dtype=torch.bool),round(s_0.shape[0]/s_1.shape[0]),dim=0)
a_2_1=torch.repeat_interleave(torch.tensor(actions_2,dtype=torch.bool),round(s_0.shape[0]/s_2.shape[0]),dim=0)

In [ ]:
print(s_0.shape)
print(s_1_1.shape)
print(s_2_1.shape)

In [ ]:
a_0=torch.tensor(actions_0,dtype=torch.bool).reshape(-1,1)
print(a_0.shape)
print(a_1_1.shape)
print(a_2_1.shape)

In [ ]:
states=torch.vstack((s_0,s_1_1,s_2_1))

In [ ]:
states.shape

In [ ]:
a_1_1=a_1_1.reshape(-1,1)
a_2_1=a_2_1.reshape(-1,1)

In [ ]:
actions=torch.vstack((a_0,a_1_1,a_2_1))
actions.shape

In [ ]:
torch.save(states,os.path.join(PATHS[4],'states','states.pt'))

In [ ]:
torch.save(actions,os.path.join(PATHS[4],'actions','actions.pt'))